The NFL is a multi-billion dollar business. There are millions of fans around the world, possibily even yourself, that invest a significant amount of time and resources to follow and watch their favorite teams and players. The teams are always working to fufill their duty to the fans. With the money they receive, they work on providing fans with the best experience possible, whether that's through giving them a great experience at the game with stadium rennovations or through building a winning team that makes fans want to tune in more. Every year, teams get around 225 million dollars to spend on players, with the money being paid through contracts. It's pretty straightforward, teams spend money on players that they think will benefit the team, and all is good, right? Obviously the answer is no, and there is a lot of nuance and subtleties that go into how much to pay players. Time and time again teams either give way too much money to certain players which ends up hurting the team, and many times teams avoid signing a certain player who ends up being a bargain for another. Then there is the question of how do we weigh which positions are worth more money. Each position has vastly different roles on a team, and therefore some are paid better than others. This however, also depends on the scheme the team has as that can affect how much they value each position. Then there are many more factors such as age and past performance that have to be accounted for. Then after all of that, even if done perfectly to the book can still end up not working. A player could not be a hard worker or lazy or had a fluke year, all of which are very hard to account for. So which are the factors you can account for? This is the question we will answer today, by looking through both basic box score predictions as well as ProFootballFocus' grading system, which breaks down the tape for every play of every player. This brings an element of context to the stats, which also can be deceiving based on what happens in a play, for example a quarterback can throw a perfect pass that gets dropped which would be considered an incompletion. We will do analysis on all different types of stats, advanced and basic, to determine which ones correlate with salary the best or how much each one correlates. This will give us a better picture of how we can predict what teams should pay a player based on their past performance and how much value in general they should add to a team. We will break down each category individually, by passing, rushing, receiving, blocking, run defense, pass rush and coverage, which will give us a nice split of positions and will allow us to differentiate the value of certain skills.

In [589]:
import pandas as pd
import numpy as np
import warnings

# filter out warnings
warnings.filterwarnings('ignore')

# get salary data 
tables = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/limit-2000/')

First we must collect data, to do this we used the python library Pandas to read the html. Since the website we are collecting from stores the data we're looking for in a table tag we can easily grab it and store it in a dataframe. This website contains information for players salary including when they signed, total value, average annual value (AAV), and information relating to guaranteed salary.

In [590]:
import re

salary_df = tables[0].rename(columns={'Player': 'player'})
players = salary_df['player']
pl = []
start_years = []
end_years = []

for player in players:
    try:
        # split up player column and extract start year end year and name
        groups = re.search(r'(.*)  .* \| (\d{4})-(\d{4}) \(FA: (\d{4})\)', player)
        pl.append(groups.groups()[0])
        start_years.append(groups.groups()[1])
        end_years.append(groups.groups()[2])
    # if regex fails dont throw error
    except:
        pl.append(None)
        start_years.append(None)
        end_years.append(None)

# insert clean data
salary_df['player'] = pl
salary_df['start_year'] = start_years
salary_df['end_year'] = end_years

# turn salaries totals into ints
def convert_to_int(v):
    try: return int(v.replace('$', '').replace(',',''))
    except: return 0


salary_df['Value'] = salary_df['Value'].apply(convert_to_int)
salary_df['AAV'] = salary_df['AAV'].apply(convert_to_int)
salary_df['Sign Bonus'] = salary_df['Sign Bonus'].apply(convert_to_int)
salary_df["G'teed @ Sign"] = salary_df["G'teed @ Sign"].apply(convert_to_int)
salary_df["Practical G'teed"] = salary_df["Practical G'teed"].apply(convert_to_int)
salary_df.fillna(0)

salary_df

,Rank,player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,1,Patrick Mahomes,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
1,2,Lamar Jackson,26,5,260000000,52000000,72500000,135000000,185000000,2023,2027
2,3,Josh Allen,25,6,258034000,43005667,16500000,100038596,150000000,2021,2028
3,4,Jalen Hurts,24,5,255000000,51000000,23294000,110000000,179399000,2023,2028
4,5,Russell Wilson,33,5,242588236,48517647,50000000,124000000,161000000,2022,2028
...,...,...,...,...,...,...,...,...,...,...,...
1995,1987,Josh Johnson,35,1,1120000,1120000,0,0,0,2022,2022
1996,1987,Vinny Curry,33,1,1120000,1120000,0,800000,800000,2022,2022
1997,1987,Oday Aboushi,30,1,1120000,1120000,0,0,0,2022,2022
1998,1987,Beau Brinkley,32,1,1120000,1120000,0,0,0,2022,2022


Now we need to get the player stats. Do do this we extracted csv files from pff.com which contain many useful statistics for each position. Again we are going to store this in a pandas dataframe to be consistent, since we will use all the datasets together. We also need to clean the data. We do this by melting the data and removing any years in which very minimal snaps were played and to only include positions we want. We want to differentiate the years because of how important it is to account for. A player having a good season 5 years ago is not nearly the same as them playing well in the last season.

In [591]:
# passing players and stats

passing_dfs = []

# passing

# 2022
passing_dfs.append(pd.read_csv('./passing/passing_summary.csv'))
passing_dfs[0]['year'] = 2022

# 2021
passing_dfs.append(pd.read_csv('./passing/passing_summary (1).csv'))
passing_dfs[1]['year'] = 2021

# 2020
passing_dfs.append(pd.read_csv('./passing/passing_summary (2).csv'))
passing_dfs[2]['year'] = 2020

# 2019
passing_dfs.append(pd.read_csv('./passing/passing_summary (3).csv'))
passing_dfs[3]['year'] = 2019

# 2018
passing_dfs.append(pd.read_csv('./passing/passing_summary (4).csv'))
passing_dfs[4]['year'] = 2018


# 2017
passing_dfs.append(pd.read_csv('./passing/passing_summary (5).csv'))
passing_dfs[5]['year'] = 2017


passing_dfs
passing_dfs[0].columns

passing_df = passing_dfs[0]
passing_df = pd.concat(passing_dfs)

# remove outliers
passing_df = passing_df[passing_df['position'] == 'QB']
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    # print(passing_df)
    pass

passing_df

passing_df = passing_df[passing_df['attempts'] > 100]

# # merge players with salary
passing_df = pd.merge(passing_df, salary_df, on='player')
passing_df

,player,player_id,position,team_name,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,avg_time_to_throw,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Patrick Mahomes,11765,QB,KC,20,78.0,685,747,7.5,2.85,...,1,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
1,Patrick Mahomes,11765,QB,KC,20,79.2,708,780,7.4,2.87,...,1,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
2,Patrick Mahomes,11765,QB,KC,18,77.1,656,705,8.5,2.91,...,1,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
3,Patrick Mahomes,11765,QB,KC,17,77.2,549,596,8.5,2.84,...,1,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
4,Patrick Mahomes,11765,QB,KC,18,77.8,595,652,9.6,2.80,...,1,24,10,450000000,45000000,10000000,63081905,141481905,2020,2031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Case Keenum,7323,QB,MIN,17,75.1,527,569,7.9,2.72,...,630,35,2,6250000,3125000,750000,4000000,4000000,2023,2024
182,David Blough,46508,QB,DET,5,68.6,159,174,8.9,2.89,...,1871,27,1,1232500,1232500,76250,76250,76250,2023,2023
183,Blaine Gabbert,6162,QB,TEN,6,66.3,98,101,7.7,2.29,...,1806,33,1,1317500,1317500,152500,1092500,1092500,2023,2023
184,Blaine Gabbert,6162,QB,ARZ,5,65.8,161,171,9.8,2.58,...,1806,33,1,1317500,1317500,152500,1092500,1092500,2023,2023


In [592]:
# rushing players and stats

rushing_dfs = []

# 2022
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
rushing_dfs.append(pd.read_csv('./rushing/rushing_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


rushing_dfs
rushing_dfs[0].columns

# merge years together
rushing_df = pd.merge(rushing_dfs[0], rushing_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
rushing_df = pd.merge(rushing_df, rushing_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
rushing_df = pd.merge(rushing_df, rushing_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
rushing_df = pd.merge(rushing_df, rushing_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
rushing_df = pd.merge(rushing_df, rushing_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
rushing_df = rushing_df[rushing_df['position'] == 'HB']
rushing_df = rushing_df[rushing_df['attempts_2022'] + rushing_df['attempts_2021'] + 
                        rushing_df['attempts_2020'] + rushing_df['attempts_2019'] + 
                        rushing_df['attempts_2018'] + rushing_df['attempts_2017'] > 100]

# merge players with salary
rushing_df = pd.merge(rushing_df, salary_df, on='player')

rushing_df

,player,player_id,position,team_name_2022,player_game_count_2022,attempts_2022,avoided_tackles_2022,breakaway_attempts_2022,breakaway_percent_2022,breakaway_yards_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Derrick Henry,10679,HB,TEN,16.0,349.0,69.0,15.0,27.6,424.0,...,142,26,4,50000000,12500000,12000000,25500000,25500000,2020,2023
1,Josh Jacobs,45953,HB,LV,17.0,339.0,90.0,15.0,23.4,386.0,...,484,25,1,10091000,10091000,0,0,0,2023,2023
2,Saquon Barkley,45791,HB,NYG,18.0,313.0,41.0,21.0,37.3,532.0,...,484,26,1,10091000,10091000,0,0,0,2023,2023
3,Nick Chubb,45783,HB,CLV,17.0,302.0,83.0,23.0,34.6,527.0,...,185,25,3,36600000,12200000,12000000,17133059,20000000,2021,2024
4,Miles Sanders,40555,HB,PHI,20.0,294.0,52.0,13.0,21.6,306.0,...,247,25,4,25400000,6350000,5900000,11000000,13000000,2023,2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Salvon Ahmed,57472,HB,MIA,7.0,17.0,2.0,0.0,0.0,0.0,...,1793,24,1,1450000,1450000,100000,300000,300000,2023,2023
65,David Johnson,9519,HB,NO,5.0,12.0,0.0,0.0,0.0,0.0,...,1987,30,1,1120000,1120000,0,0,0,2022,2022
66,Myles Gaskin,45914,HB,MIA,2.0,10.0,3.0,0.0,0.0,0.0,...,1871,26,1,1232500,1232500,0,0,0,2023,2023
67,Ameer Abdullah,9487,HB,LV,15.0,4.0,0.0,0.0,0.0,0.0,...,1724,29,1,1750000,1750000,200000,500000,500000,2023,2023


In [593]:
# receiving players and stats

receiving_dfs = []

# 2022
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
receiving_dfs.append(pd.read_csv('./receiving/receiving_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


receiving_dfs
receiving_dfs[0].columns

# merge years together
receiving_df = pd.merge(receiving_dfs[0], receiving_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
receiving_df = pd.merge(receiving_df, receiving_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
receiving_df = pd.merge(receiving_df, receiving_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
receiving_df = pd.merge(receiving_df, receiving_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
receiving_df = pd.merge(receiving_df, receiving_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
receiving_df = receiving_df[receiving_df['targets_2022'] + receiving_df['targets_2021'] + 
                        receiving_df['targets_2020'] + receiving_df['targets_2019'] + 
                        receiving_df['targets_2018'] + receiving_df['targets_2017'] > 100]

# merge players with salary
receiving_df = pd.merge(receiving_df, salary_df, on='player')

receiving_df

,player,player_id,position,team_name_2022,player_game_count_2022,avg_depth_of_target_2022,avoided_tackles_2022,caught_percent_2022,contested_catch_rate_2022,contested_receptions_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Justin Jefferson,61398,WR,MIN,18.0,10.7,11.0,73.0,55.0,22.0,...,418,21,4,13122805,3280701,7103856,13122805,13122805,2020,2024
1,Travis Kelce,7844,TE,KC,20.0,7.5,25.0,75.3,45.5,10.0,...,115,30,4,57250000,14312500,0,20750000,22750000,2020,2025
2,Tyreek Hill,10799,WR,MIA,18.0,12.6,12.0,69.6,50.0,13.0,...,19,28,4,120000000,30000000,25500000,52535000,72200000,2022,2026
3,Davante Adams,8688,WR,LV,17.0,12.8,16.0,59.5,44.1,15.0,...,14,29,5,140000000,28000000,19250000,22750000,65670000,2022,2026
4,CeeDee Lamb,61570,WR,DAL,19.0,10.3,16.0,72.5,46.7,14.0,...,393,21,4,14010012,3502503,7749100,14010012,14010012,2020,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Jalen Guyton,34164,WR,LAC,3.0,28.3,0.0,50.0,50.0,1.0,...,1871,25,1,1232500,1232500,76000,76000,76000,2023,2023
174,Tim Patrick,12087,WR,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,202,27,3,34000000,11333333,3000000,11500000,18500000,2021,2024
175,Calvin Ridley,48262,WR,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,479,23,4,10900711,2725178,6007790,8454251,8454251,2018,2023
176,Rashard Higgins,10806,WR,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1937,27,1,1187500,1187500,152500,152500,152500,2022,2022


In [594]:
# offense_blocking players and stats

offense_blocking_dfs = []

# 2022
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
offense_blocking_dfs.append(pd.read_csv('./offense_blocking/offense_blocking (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


offense_blocking_dfs
offense_blocking_dfs[0].columns

# merge years together
offense_blocking_df = pd.merge(offense_blocking_dfs[0], offense_blocking_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
offense_blocking_df = pd.merge(offense_blocking_df, offense_blocking_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
offense_blocking_df = offense_blocking_df[(offense_blocking_df['position'] == 'C') | 
                                          (offense_blocking_df['position'] == 'G') | 
                                          (offense_blocking_df['position'] == 'T')]
offense_blocking_df = offense_blocking_df[offense_blocking_df['snap_counts_block_2022'] + offense_blocking_df['snap_counts_block_2021'] + 
                        offense_blocking_df['snap_counts_block_2020'] + offense_blocking_df['snap_counts_block_2019'] + 
                        offense_blocking_df['snap_counts_block_2018'] + offense_blocking_df['snap_counts_block_2017'] > 100]

# merge players with salary
offense_blocking_df = pd.merge(offense_blocking_df, salary_df, on='player')

offense_blocking_df

,player,player_id,position,team_name_2022,player_game_count_2022,block_percent_2022,declined_penalties_2022,franchise_id_2022,grades_offense_2022,grades_pass_block_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Jason Kelce,6343,C,PHI,20.0,100.0,1.0,24.0,89.4,83.3,...,386,35,1,14250000,14250000,10335000,14250000,14250000,2023,2023
1,Isaac Seumalo,10713,G,PHI,20.0,100.0,3.0,24.0,72.7,78.2,...,256,29,3,24000000,8000000,6950000,6950000,6950000,2023,2025
2,Creed Humphrey,59996,C,KC,20.0,100.0,1.0,16.0,89.9,81.3,...,670,21,4,5565208,1391302,1407424,2320388,2320388,2021,2024
3,Orlando Brown Jr.,46227,T,KC,20.0,100.0,1.0,16.0,75.4,76.8,...,93,26,4,64092000,16023000,31100000,31100000,31100000,2023,2026
4,Cordell Volson,30614,G,CIN,19.0,100.0,1.0,7.0,53.7,52.7,...,873,23,4,4316884,1079221,656884,656884,656884,2022,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Trystan Colon-Castillo,41117,C,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1761,24,1,1660000,1660000,250000,250000,250000,2023,2023
316,Chuma Edoga,46232,T,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1871,25,1,1232500,1232500,152500,1092500,1092500,2023,2023
317,Aaron Stinnie,28653,G,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1795,28,1,1400000,1400000,0,150000,150000,2023,2023
318,Mekhi Becton,59818,T,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,327,21,4,18446048,4611512,10975308,18446048,18446048,2020,2024


Pffs data does not break down by position entirely especially on defense. We need pass rush and run defense to be involved for d lineman and edge rushers, we need linebackers to have run defense and coverage stats, and we need to split up safeties and cornerbacks in regards to coverage data.

In [595]:
# run_defense players and stats

run_defense_dfs = []

# 2022
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
run_defense_dfs.append(pd.read_csv('./run_defense/run_defense_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


run_defense_dfs
run_defense_dfs[0].columns

# merge years together
run_defense_df = pd.merge(run_defense_dfs[0], run_defense_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
run_defense_df = pd.merge(run_defense_df, run_defense_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
run_defense_df = run_defense_df[run_defense_df['snap_counts_run_2022'] + run_defense_df['snap_counts_run_2021'] + 
                        run_defense_df['snap_counts_run_2020'] + run_defense_df['snap_counts_run_2019'] + 
                        run_defense_df['snap_counts_run_2018'] + run_defense_df['snap_counts_run_2017'] > 100]

# merge players with salary
run_defense_df = pd.merge(run_defense_df, salary_df, on='player')

run_defense_df

# pass_rush players and stats

pass_rush_dfs = []

# 2022
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
pass_rush_dfs.append(pd.read_csv('./pass_rush/pass_rush_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


pass_rush_dfs
pass_rush_dfs[0].columns

# merge years together
pass_rush_df = pd.merge(pass_rush_dfs[0], pass_rush_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
pass_rush_df = pd.merge(pass_rush_df, pass_rush_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
pass_rush_df = pass_rush_df[pass_rush_df['snap_counts_pass_rush_2022'] + pass_rush_df['snap_counts_pass_rush_2021'] + 
                        pass_rush_df['snap_counts_pass_rush_2020'] + pass_rush_df['snap_counts_pass_rush_2019'] + 
                        pass_rush_df['snap_counts_pass_rush_2018'] + pass_rush_df['snap_counts_pass_rush_2017'] > 100]

# merge players with salary
pass_rush_df = pd.merge(pass_rush_df, salary_df, on='player')

# merge run defense and pass rush
front_7_df = pd.merge(pass_rush_df, run_defense_df, on=['player_id', 'player', 'position']).fillna(-1)

In [596]:
# interior defensive line
idl_df = front_7_df[front_7_df['position'] == 'DI']
idl_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
0,Calais Campbell,4364,DI,BLT,15.0,2.0,1.0,3.0,71.7,12.0,...,598,36,1,7000000,7000000,3000000,7000000,7000000,2023,2023
2,Ndamukong Suh,5527,DI,PHI,11.0,0.0,0.0,24.0,55.5,3.0,...,1667,35,1,2000000,2000000,250000,250000,250000,2022,2022
3,Tyson Alualu,5535,DI,PIT,17.0,1.0,0.0,25.0,58.5,2.0,...,674,33,2,5500000,2750000,1925000,1925000,1925000,2021,2022
7,Linval Joseph,5571,DI,PHI,11.0,0.0,0.0,24.0,60.2,0.0,...,1667,34,1,2000000,2000000,250000,250000,250000,2022,2022
12,Cameron Heyward,6183,DI,PIT,17.0,2.0,0.0,25.0,78.5,10.0,...,91,31,4,65600000,16400000,17500000,20250000,26250000,2020,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,Sheldon Day,10737,DI,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1959,28,1,1165000,1165000,0,0,0,2023,2023
363,Vincent Taylor,11949,DI,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1937,28,1,1187500,1187500,152500,275000,275000,2022,2022
364,Maurice Hurst,38563,DI,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1868,27,1,1242000,1242000,0,0,0,2023,2023
366,Levi Onwuzurike,43679,DI,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,554,23,4,8148893,2037223,3286468,4316872,4316872,2021,2024


In [597]:
# edge rushers
edge_df = front_7_df[front_7_df['position'] == 'ED']
edge_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
1,Calais Campbell,4364,ED,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,598,36,1,7000000,7000000,3000000,7000000,7000000,2023,2023
4,Brandon Graham,5538,ED,PHI,20.0,1.0,0.0,24.0,89.8,4.0,...,744,34,1,5000000,5000000,3835000,5000000,5000000,2023,2023
5,Jason Pierre-Paul,5540,ED,BLT,15.0,4.0,1.0,3.0,56.0,0.0,...,1803,33,1,1350000,1350000,150000,1150000,1150000,2022,2022
6,Jerry Hughes,5556,ED,HST,17.0,0.0,2.0,13.0,71.3,0.0,...,487,33,2,10000000,5000000,2500000,4500000,4500000,2022,2023
8,Carlos Dunlap,5579,ED,KC,20.0,8.0,0.0,16.0,61.0,11.0,...,1249,33,1,3000000,3000000,1880000,3000000,3000000,2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Kayvon Thibodeaux,98444,ED,NYG,16.0,5.0,0.0,21.0,67.3,12.0,...,217,21,4,31339038,7834760,19972028,31339038,31339038,2022,2026
354,Travon Walker,98940,ED,JAX,17.0,0.0,3.0,15.0,58.6,8.0,...,180,21,4,37372621,9343155,24360088,37372621,37372621,2022,2026
355,Sam Williams,99106,ED,DAL,17.0,0.0,2.0,9.0,70.9,7.0,...,633,23,4,6224251,1556063,1706728,2694649,2694649,2022,2025
362,Stephen Weatherly,10861,ED,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1934,28,1,1200000,1200000,150000,650000,650000,2022,2022


In [598]:
# linebackers
lb_df = front_7_df[front_7_df['position'] == 'LB']
lb_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank_y,Signed Age_y,Yrs_y,Value_y,AAV_y,Sign Bonus_y,G'teed @ Sign_y,Practical G'teed_y,start_year_y,end_year_y
18,Bobby Wagner,7050,LB,LA,17.0,0.0,0.0,26.0,79.6,3.0,...,674,32,1,5500000,5500000,1250000,5500000,5500000,2023,2023
19,Lavonte David,7060,LB,TB,18.0,0.0,0.0,30.0,65.0,1.0,...,825,33,1,4500000,4500000,3335000,3335000,3335000,2023,2023
21,Demario Davis,7079,LB,NO,17.0,0.0,0.0,20.0,85.9,2.0,...,240,31,3,27000000,9000000,11000000,18350000,18350000,2020,2024
24,Jon Bostic,7831,LB,WAS,9.0,0.0,0.0,32.0,50.6,0.0,...,1987,31,1,1120000,1120000,0,0,0,2022,2022
27,A.J. Klein,7930,LB,BUF,5.0,0.0,0.0,4.0,60.7,0.0,...,1806,31,1,1317500,1317500,100000,100000,100000,2023,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,Zaven Collins,77190,LB,ARZ,16.0,2.0,1.0,1.0,65.5,2.0,...,380,21,4,14690257,3672564,8043824,14690257,14690257,2021,2025
337,Nick Bolton,81322,LB,KC,20.0,1.0,0.0,16.0,61.6,5.0,...,657,20,4,5834032,1458508,1602932,2528115,2528115,2021,2024
339,Micah Parsons,81360,LB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,345,22,4,17079793,4269948,9781668,17079793,17079793,2021,2025
360,Neville Hewitt,9836,LB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1686,28,1,1950000,1950000,350000,850000,850000,2022,2022


In [599]:
# defense_coverage players and stats

defense_coverage_dfs = []

# 2022
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary.csv').add_suffix('_2022').rename(
    columns={'player_2022': 'player', 'player_id_2022': 'player_id', 'position_2022': 'position',}))

# 2021
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (1).csv').add_suffix('_2021').rename(
    columns={'player_2021': 'player', 'player_id_2021': 'player_id', 'position_2021': 'position', }))

# 2020
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (2).csv').add_suffix('_2020').rename(
    columns={'player_2020': 'player', 'player_id_2020': 'player_id', 'position_2020': 'position', }))

# 2019
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (3).csv').add_suffix('_2019').rename(
    columns={'player_2019': 'player', 'player_id_2019': 'player_id', 'position_2019': 'position', }))

# 2018
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (4).csv').add_suffix('_2018').rename(
    columns={'player_2018': 'player', 'player_id_2018': 'player_id', 'position_2018': 'position', }))

# 2017
defense_coverage_dfs.append(pd.read_csv('./defense_coverage/defense_coverage_summary (5).csv').add_suffix('_2017').rename(
    columns={'player_2017': 'player', 'player_id_2017': 'player_id', 'position_2017': 'position'}))


defense_coverage_dfs
defense_coverage_dfs[0].columns

# merge years together
defense_coverage_df = pd.merge(defense_coverage_dfs[0], defense_coverage_dfs[1], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[2], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[3], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[4], on=['player_id', 'player', 'position'], how='outer').fillna(-1)
defense_coverage_df = pd.merge(defense_coverage_df, defense_coverage_dfs[5], on=['player_id', 'player', 'position'], how='outer').fillna(-1)

# remove outliers
defense_coverage_df = defense_coverage_df[defense_coverage_df['snap_counts_coverage_2022'] + defense_coverage_df['snap_counts_coverage_2021'] + 
                        defense_coverage_df['snap_counts_coverage_2020'] + defense_coverage_df['snap_counts_coverage_2019'] + 
                        defense_coverage_df['snap_counts_coverage_2018'] + defense_coverage_df['snap_counts_coverage_2017'] > 100]

# merge players with salary
defense_coverage_df = pd.merge(defense_coverage_df, salary_df, on='player')

defense_coverage_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Jason Pierre-Paul,5540,ED,BLT,15.0,0.0,2.4,100.0,4.9,3.0,...,1803,33,1,1350000,1350000,150000,1150000,1150000,2022,2022
1,Kareem Jackson,5545,S,DEN,17.0,5.0,8.0,69.0,99.7,22.9,...,1667,33,1,2000000,2000000,700000,2000000,2000000,2022,2022
2,Kareem Jackson,5545,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1667,33,1,2000000,2000000,700000,2000000,2000000,2022,2022
3,Jerry Hughes,5556,ED,HST,17.0,0.0,-1.0,-1.0,0.8,-1.0,...,487,33,2,10000000,5000000,2500000,4500000,4500000,2022,2023
4,Carlos Dunlap,5579,ED,KC,20.0,1.0,8.5,100.0,4.5,10.5,...,1249,33,1,3000000,3000000,1880000,3000000,3000000,2022,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Marcus Allen,49263,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1614,25,1,2540000,2540000,0,0,0,2022,2022
433,Cody Davis,8458,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1654,33,1,2200000,2200000,0,500000,500000,2023,2023
434,Jamal Agnew,11920,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,386,25,3,14250000,4750000,3500000,4000000,4000000,2021,2023
435,Leonard Johnson,7385,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1305,24,3,2695000,898333,0,0,0,2023,2025


In [600]:
# linebackers
full_lb_df = pd.merge(lb_df, defense_coverage_df[defense_coverage_df['position'] == 'LB'], on=['player_id', 'player', 'position'])
full_lb_df

,player,player_id,position,team_name_2022_x,player_game_count_2022_x,batted_passes_2022,declined_penalties_2022_x,franchise_id_2022_x,grades_pass_rush_defense_2022_x,hits_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
0,Bobby Wagner,7050,LB,LA,17.0,0.0,0.0,26.0,79.6,3.0,...,674,32,1,5500000,5500000,1250000,5500000,5500000,2023,2023
1,Lavonte David,7060,LB,TB,18.0,0.0,0.0,30.0,65.0,1.0,...,825,33,1,4500000,4500000,3335000,3335000,3335000,2023,2023
2,Demario Davis,7079,LB,NO,17.0,0.0,0.0,20.0,85.9,2.0,...,240,31,3,27000000,9000000,11000000,18350000,18350000,2020,2024
3,Jon Bostic,7831,LB,WAS,9.0,0.0,0.0,32.0,50.6,0.0,...,1987,31,1,1120000,1120000,0,0,0,2022,2022
4,A.J. Klein,7930,LB,BUF,5.0,0.0,0.0,4.0,60.7,0.0,...,1806,31,1,1317500,1317500,100000,100000,100000,2023,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Zaven Collins,77190,LB,ARZ,16.0,2.0,1.0,1.0,65.5,2.0,...,380,21,4,14690257,3672564,8043824,14690257,14690257,2021,2025
71,Nick Bolton,81322,LB,KC,20.0,1.0,0.0,16.0,61.6,5.0,...,657,20,4,5834032,1458508,1602932,2528115,2528115,2021,2024
72,Micah Parsons,81360,LB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,345,22,4,17079793,4269948,9781668,17079793,17079793,2021,2025
73,Neville Hewitt,9836,LB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1686,28,1,1950000,1950000,350000,850000,850000,2022,2022


In [601]:
# cornerbacks
cb_df = defense_coverage_df[defense_coverage_df['position'] == 'CB']
cb_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
2,Kareem Jackson,5545,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1667,33,1,2000000,2000000,700000,2000000,2000000,2022,2022
6,Patrick Peterson,6157,CB,MIN,18.0,2.0,12.1,60.0,99.3,14.6,...,394,32,2,14000000,7000000,5850000,5850000,5850000,2023,2024
10,Stephon Gilmore,7016,CB,IND,16.0,2.0,9.3,63.4,100.0,10.9,...,308,31,2,20000000,10000000,4000000,9510000,14000000,2022,2023
19,Justin Bethel,7654,CB,MIA,5.0,0.0,8.7,60.0,100.0,7.6,...,1806,32,1,1317500,1317500,75000,75000,75000,2023,2023
20,Darius Slay,7817,CB,PHI,20.0,4.0,11.6,56.6,100.0,14.6,...,160,31,3,42000000,14000000,10185000,24500000,24500000,2023,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Chris Jones,49339,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,58,26,4,80000000,20000000,0,37626000,60000000,2020,2023
425,Isaac Yiadom,49355,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1871,27,1,1232500,1232500,0,0,0,2023,2023
428,Ifeatu Melifonwu,56203,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,791,22,4,4785852,1196463,840620,840620,840620,2021,2024
434,Jamal Agnew,11920,CB,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,386,25,3,14250000,4750000,3500000,4000000,4000000,2021,2023


In [602]:
# Safeties
s_df = defense_coverage_df[defense_coverage_df['position'] == 'S']
s_df

,player,player_id,position,team_name_2022,player_game_count_2022,assists_2022,avg_depth_of_target_2022,catch_rate_2022,coverage_percent_2022,coverage_snaps_per_reception_2022,...,Rank,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed,start_year,end_year
1,Kareem Jackson,5545,S,DEN,17.0,5.0,8.0,69.0,99.7,22.9,...,1667,33,1,2000000,2000000,700000,2000000,2000000,2022,2022
16,Michael Thomas,7279,S,CIN,3.0,0.0,-1.0,-1.0,88.2,-1.0,...,487,30,1,10000000,10000000,5000000,10000000,10000000,2023,2024
17,Michael Thomas,7279,S,CIN,3.0,0.0,-1.0,-1.0,88.2,-1.0,...,1959,32,1,1165000,1165000,0,0,0,2023,2023
18,Harrison Smith,7641,S,MIN,15.0,5.0,10.0,80.0,97.8,16.4,...,96,32,4,64000000,16000000,9579410,14179410,26379410,2021,2025
22,Tyrann Mathieu,7850,S,NO,17.0,5.0,5.7,67.4,96.9,20.4,...,236,29,3,28300000,9433333,9500000,18000000,18000000,2022,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,Sheldrick Redwine,50458,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1655,26,2,2135000,1067500,0,0,0,2023,2024
427,Armani Watts,51264,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1937,26,1,1187500,1187500,152500,402500,402500,2022,2022
432,Marcus Allen,49263,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1614,25,1,2540000,2540000,0,0,0,2022,2022
433,Cody Davis,8458,S,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1654,33,1,2200000,2200000,0,500000,500000,2023,2023


In [604]:
print(passing_df.isin([np.nan, np.inf, -np.inf]).sum())

for p in passing_df.iterrows():
    print(p)
    # print(p)
print(passing_df['grades_run'].to_string())


player                   0
player_id                0
position                 0
team_name                0
player_game_count        0
accuracy_percent         0
aimed_passes             0
attempts                 0
avg_depth_of_target      0
avg_time_to_throw        0
bats                     0
big_time_throws          0
btt_rate                 0
completion_percent       0
completions              0
declined_penalties       0
def_gen_pressures        0
drop_rate                0
dropbacks                0
drops                    0
first_downs              0
franchise_id             0
grades_hands_fumble      0
grades_offense           0
grades_pass              0
grades_run               0
hit_as_threw             0
interceptions            0
passing_snaps            0
penalties                0
pressure_to_sack_rate    0
qb_rating                0
sack_percent             0
sacks                    0
scrambles                0
spikes                   0
thrown_aways             0
t

In [605]:
passing_df.columns

Index(['player', 'player_id', 'position', 'team_name', 'player_game_count',
       'accuracy_percent', 'aimed_passes', 'attempts', 'avg_depth_of_target',
       'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate',
       'completion_percent', 'completions', 'declined_penalties',
       'def_gen_pressures', 'drop_rate', 'dropbacks', 'drops', 'first_downs',
       'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass',
       'grades_run', 'hit_as_threw', 'interceptions', 'passing_snaps',
       'penalties', 'pressure_to_sack_rate', 'qb_rating', 'sack_percent',
       'sacks', 'scrambles', 'spikes', 'thrown_aways', 'touchdowns',
       'turnover_worthy_plays', 'twp_rate', 'yards', 'ypa', 'year', 'Rank',
       'Signed Age', 'Yrs', 'Value', 'AAV', 'Sign Bonus', 'G'teed @ Sign',
       'Practical G'teed', 'start_year', 'end_year'],
      dtype='object')

Now we need to explore our data and understand it. This is where we analyze the features of our dataset and start to determine which ones are more valueable for our hypothesis. To do this we will use single vector decomposition. We will import svds from the scipy linear algebra library and apply it to our data. We need to first remove any irrelevant data from ourdatasets such as player_id, franchise_id and team and handle any missing data. For missing data we replaced it with the mean from that column. Since pretty much all missing data has already been dropped this will not affect the analysis here much.

In [630]:
import numpy as np
from scipy.sparse.linalg import svds

# passing_df = passing_df.fillna(passing_df.mean())
numeric_cols = passing_df.select_dtypes(include=[np.number]).columns.tolist()

passing_df[numeric_cols] = passing_df[numeric_cols].replace(-1, passing_df[numeric_cols].mean()).fillna(passing_df[numeric_cols].mean())

# Extract the matrix of numerical features you want to perform SVD on
features = passing_df.select_dtypes(include=[np.number])

features = features.drop('player_id', axis=1)
features = features.drop('franchise_id', axis=1)
features = features.drop('Rank', axis=1)
cols = len(features.columns) -1 
features
# Perform SVD on the matrix
U, S, Vt = svds(features.to_numpy(), k=cols)

# Check the shape of the SVD factors
print('Shape of U:', U.shape)
print('Shape of S:', S.shape)
print('Shape of Vt:', Vt.shape)
Vt[cols-1]


Shape of U: (186, 44)
Shape of S: (44,)
Shape of Vt: (44, 45)


array([-5.64571315e-08, -2.79650806e-07, -1.70959119e-06, -1.85208692e-06,
       -3.26887517e-08, -1.06375890e-08, -2.94353690e-08, -9.97025705e-08,
       -1.83384846e-08, -2.41057977e-07, -1.20798245e-06, -6.83009503e-10,
       -7.18768462e-07, -2.45545067e-08, -2.10294736e-06, -8.52562027e-08,
       -7.25663400e-07, -2.20555464e-07, -2.91801136e-07, -2.81633050e-07,
       -2.65807284e-07, -1.92443599e-08, -3.63278127e-08, -2.23706112e-06,
       -1.43650775e-08, -6.80144375e-08, -3.57041682e-07, -2.34761040e-08,
       -1.27121222e-07, -1.18874767e-07, -7.48578091e-09, -8.63302202e-08,
       -9.75485097e-08, -6.89154370e-08, -1.19072056e-08, -1.39482704e-05,
       -2.79065689e-08, -7.53276643e-06, -1.04141391e-07, -1.81416226e-08,
       -7.63538184e-01, -1.50318261e-01, -1.31388325e-01, -3.53187851e-01,
       -5.02403534e-01])

These values indicate how much correlation there are between all the data. Each element in the vector represents its respective column. If the value is closer to 0 that means there is high correlation between that feature and the dataset. Now we will find and list out the features themselves in order from most correlated to least to give us a good picture of the value for our features.

In [635]:
sorted_feature_weights = sorted(Vt[cols-1])
for i, w in enumerate(sorted_feature_weights):
    print(f'{i+1}: {list(features.columns)[np.where(Vt[cols-1] == w)[0][0]]}')

1: Value
2: Practical G'teed
3: G'teed @ Sign
4: AAV
5: Sign Bonus
6: yards
7: year
8: passing_snaps
9: dropbacks
10: attempts
11: aimed_passes
12: completions
13: first_downs
14: def_gen_pressures
15: qb_rating
16: grades_offense
17: grades_pass
18: accuracy_percent
19: grades_run
20: completion_percent
21: grades_hands_fumble
22: sacks
23: scrambles
24: Signed Age
25: big_time_throws
26: touchdowns
27: thrown_aways
28: drops
29: turnover_worthy_plays
30: pressure_to_sack_rate
31: player_game_count
32: interceptions
33: avg_depth_of_target
34: bats
35: ypa
36: drop_rate
37: sack_percent
38: hit_as_threw
39: btt_rate
40: Yrs
41: penalties
42: twp_rate
43: avg_time_to_throw
44: spikes
45: declined_penalties
